<h1 align=center>Dr Appointments</h1>

In [18]:
import os
from dotenv import load_dotenv
load_dotenv()


from pydantic import BaseModel, Field, field_validator
import re
import pandas as pd
from typing import  Literal,List,Any

from langchain_core.tools import tool


In [ ]:
os.environ["GROQ_API_KEY"]= os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [4]:
from langchain_groq import ChatGroq

llm=ChatGroq(model_name="deepseek-r1-distill-llama-70b")

In [5]:
results = llm.invoke("tell me about groq in a few sentences")
print(results.content)

<think>
Okay, so I need to tell the user about Groq in a few sentences. From what I know, Groq is a company that makes AI chips. They were founded a few years back, maybe around 2016 or so. Their main focus is on creating specialized hardware for machine learning, which is a big deal because AI requires a lot of processing power. 

I remember hearing that Groq was started by some people from Google, which makes sense because Google is big on AI. The founders probably saw a need for more efficient chips since general-purpose CPUs aren't the best for AI tasks. So, Groq must be trying to solve that problem by offering something more optimized.

Their chips are probably designed to handle things like matrix multiplication, which is a core part of neural networks. Maybe they use some kind of tensor processing units, similar to what Google does with their TPUs. That would make their chips more efficient for AI workloads compared to regular GPUs or CPUs.

I think Groq has been getting some at

In [10]:
class DateTimeModel(BaseModel):
    date: str = Field(description="Properly formatted date", pattern=r'^\d{2}-\d{2}-\d{4} \d{2}:\d{2}$')

    @field_validator("date")
    def check_format_date(cls, v):
        if not re.match(r'^\d{2}-\d{2}-\d{4} \d{2}:\d{2}$', v):  # Ensures 'DD-MM-YYYY HH:MM' format
            raise ValueError("The date should be in format 'DD-MM-YYYY HH:MM'")
        return v

In [11]:
class DateModel(BaseModel):
    date: str = Field(description="Properly formatted date", pattern=r'^\d{2}-\d{2}-\d{4}$')

    @field_validator("date")
    def check_format_date(cls, v):
        if not re.match(r'^\d{2}-\d{2}-\d{4}$', v):  # Ensures DD-MM-YYYY format
            raise ValueError("The date must be in the format 'DD-MM-YYYY'")
        return v

In [12]:
class IdentificationNumberModel(BaseModel):
    id: int = Field(description="Identification number (7 or 8 digits long)")
    @field_validator("id")
    def check_format_id(cls, v):
        if not re.match(r'^\d{7,8}$', str(v)):  # Convert to string before matching
            raise ValueError("The ID number should be a 7 or 8-digit number")
        return v

In [16]:
print(DateTimeModel(date="22-12-2023 09:45"))

print(DateModel(date="03-09-2024"))

print(IdentificationNumberModel(id="78797516"))

date='22-12-2023 09:45'
date='03-09-2024'
id=78797516


In [19]:
df = pd.read_csv("availability.csv")
df.head()

,date_slot,specialization,doctor_name,is_available,patient_to_attend
0,05-08-2024 08:00,general_dentist,john doe,True,NaN
1,05-08-2024 08:30,general_dentist,john doe,False,1000082.0
2,05-08-2024 09:00,general_dentist,john doe,False,1000048.0
3,05-08-2024 09:30,general_dentist,john doe,False,1000036.0
4,05-08-2024 10:00,general_dentist,john doe,False,1000024.0


In [33]:
df['time'] = df['date_slot'].apply(lambda input: input.split(' ')[-1])
df.time

0       08:00
1       08:30
2       09:00
3       09:30
4       10:00
        ...  
4275    14:30
4276    15:00
4277    15:30
4278    16:00
4279    16:30
Name: time, Length: 4280, dtype: object

In [22]:
dates = df['date_slot'].apply(lambda input: input.split(' ')[0])
dates

0       05-08-2024
1       05-08-2024
2       05-08-2024
3       05-08-2024
4       05-08-2024
           ...    
4275    03-09-2024
4276    03-09-2024
4277    03-09-2024
4278    03-09-2024
4279    03-09-2024
Name: date_slot, Length: 4280, dtype: object

In [34]:
(df[(dates == "05-08-2024") & (df.doctor_name == "john doe") & (df.is_available == True)]['time'])

0    08:00
Name: time, dtype: object

In [40]:
@tool
def check_availability_by_doctor(desired_date:DateModel, doctor_name:Literal['kevin anderson','robert martinez','susan davis','daniel miller','sarah wilson','michael green','lisa brown','jane smith','emily johnson','john doe']):
    """
    Checking the database if we have availability for the specific doctor.
    The parameters should be mentioned by the user in the query
    """
    df = pd.read_csv(r"availability.csv")
    
    df['date_slot_time'] = df['date_slot'].apply(lambda input: input.split(' ')[-1])
    
    rows = list(df[(df['date_slot'].apply(lambda input: input.split(' ')[0]) == desired_date.date)&(df['doctor_name'] == doctor_name)&(df['is_available'] == True)]['date_slot_time'])

    if len(rows) == 0:
        output = "No availability in the entire day"
    else:
        output = f'This availability for {desired_date.date}\n'
        output += "Available slots: " + ', '.join(rows)

    return output

In [44]:
print(check_availability_by_doctor.invoke({"desired_date": DateModel(date="03-09-2024"), "doctor_name": "kevin anderson"}))

This availability for 03-09-2024
Available slots: 08:00, 08:30, 11:30, 12:00, 12:30, 13:00, 14:00, 14:30, 15:00, 15:30, 16:00, 16:30


In [45]:
print(check_availability_by_doctor.invoke({"desired_date": DateModel(date="05-08-2024"), "doctor_name": "kevin anderson"}))

No availability in the entire day


In [46]:
@tool
def check_availability_by_specialization(desired_date:DateModel, specialization:Literal["general_dentist", "cosmetic_dentist", "prosthodontist", "pediatric_dentist","emergency_dentist","oral_surgeon","orthodontist"]):
    """
    Checking the database if we have availability for the specific specialization.
    The parameters should be mentioned by the user in the query
    """
    #Dummy data
    df = pd.read_csv(r"availability.csv")
    df['date_slot_time'] = df['date_slot'].apply(lambda input: input.split(' ')[-1])
    rows = df[(df['date_slot'].apply(lambda input: input.split(' ')[0]) == desired_date.date) & (df['specialization'] == specialization) & (df['is_available'] == True)].groupby(['specialization', 'doctor_name'])['date_slot_time'].apply(list).reset_index(name='available_slots')

    if len(rows) == 0:
        output = "No availability in the entire day"
    else:
        def convert_to_am_pm(time_str):
            # Split the time string into hours and minutes
            time_str = str(time_str)
            hours, minutes = map(int, time_str.split(":"))
            
            # Determine AM or PM
            period = "AM" if hours < 12 else "PM"
            
            # Convert hours to 12-hour format
            hours = hours % 12 or 12
            
            # Format the output
            return f"{hours}:{minutes:02d} {period}"
        output = f'This availability for {desired_date.date}\n'
        for row in rows.values:
            output += row[1] + ". Available slots: \n" + ', \n'.join([convert_to_am_pm(value)for value in row[2]])+'\n'

    return output

In [47]:
print(check_availability_by_specialization.invoke({"desired_date": DateModel(date="03-09-2024"), "specialization": "orthodontist"}))

This availability for 03-09-2024
kevin anderson. Available slots: 
8:00 AM, 
8:30 AM, 
11:30 AM, 
12:00 PM, 
12:30 PM, 
1:00 PM, 
2:00 PM, 
2:30 PM, 
3:00 PM, 
3:30 PM, 
4:00 PM, 
4:30 PM

